# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,broken hill,-31.9500,141.4333,21.16,76,99,7.75,AU,1732395525
1,1,edinburgh of the seven seas,-37.0676,-12.3116,12.38,63,24,6.92,SH,1732395498
2,2,bom jesus do galho,-19.8289,-42.3161,18.70,92,100,1.51,BR,1732395792
3,3,yellowknife,62.4560,-114.3525,-13.71,78,100,1.54,CA,1732395793
4,4,mount gambier,-37.8333,140.7667,15.84,92,100,2.85,AU,1732395795


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_data_df.count()

City_ID       575
City          575
Lat           575
Lng           575
Max Temp      575
Humidity      575
Cloudiness    575
Wind Speed    575
Country       570
Date          575
dtype: int64

In [ ]:
missing_countries = city_data_df[city_data_df['Country'].isnull()]
missing_countries

In [4]:
city_data_df.dropna(subset=['Country'], inplace=True)

In [5]:
city_data_df.count()

City_ID       570
City          570
Lat           570
Lng           570
Max Temp      570
Humidity      570
Cloudiness    570
Wind Speed    570
Country       570
Date          570
dtype: int64

In [14]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [35]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
    
    
)

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
filter_ideal_weather=(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']< 27) &(city_data_df['Wind Speed']<4.5)&(city_data_df['Cloudiness']==0)
ideal_city_data_df = city_data_df.loc[filter_ideal_weather]
# Drop any rows with null values
# Already dropped rows with null values

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Humidity_scaled
13,13,mecca,21.4267,39.8261,25.69,64,0,1.97,SA,1732395806,0.635955
72,72,villa gonzalez ortega,22.5000,-101.9167,22.67,15,0,2.21,MX,1732395883,0.140449
127,127,holualoa,19.6228,-155.9522,24.90,77,0,3.60,US,1732395945,0.767416
201,201,mramani,-12.3464,44.5286,25.73,83,0,2.00,KM,1732396034,0.828090
308,308,amayuca,18.7000,-98.7667,24.45,25,0,1.41,MX,1732399895,0.241573
343,343,ouani,-12.1322,44.4258,26.78,79,0,3.40,KM,1732399939,0.787640
372,372,al wajh,26.2455,36.4525,23.97,60,0,2.06,SA,1732400055,0.595506
378,378,lihue,21.9789,-159.3672,26.52,65,0,3.60,US,1732400073,0.646067
430,430,gunjur,13.2019,-16.7339,25.04,88,0,2.57,GM,1732400141,0.878652
444,444,crane,31.3974,-102.3501,24.73,20,0,4.16,US,1732400158,0.191011


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_city_data_df[['City','Country','Lat','Lng','Humidity']].copy()#Xpert code help

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,mecca,SA,21.4267,39.8261,64,
72,villa gonzalez ortega,MX,22.5000,-101.9167,15,
127,holualoa,US,19.6228,-155.9522,77,
201,mramani,KM,-12.3464,44.5286,83,
308,amayuca,MX,18.7000,-98.7667,25,
343,ouani,KM,-12.1322,44.4258,79,
372,al wajh,SA,26.2455,36.4525,60,
378,lihue,US,21.9789,-159.3672,65,
430,gunjur,GM,13.2019,-16.7339,88,
444,crane,US,31.3974,-102.3501,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
category= "accommodation.hotel"
limit = 20
params = {
    "categories":category,
    "limit": limit,
    "apiKey":geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
   latitude = row['Lat']
   longitude=row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
   params["filter"] = f"circle:{longitude},{latitude},{radius}"
   params["bias"] = f"proximity:{longitude},{latitude}" 

    

    # Set base URL
   base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
   name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
   name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
   except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
   print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mecca - nearest hotel: Rehab Al Marwa Hotel 1
villa gonzalez ortega - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
mramani - nearest hotel: Loulou Hotel
amayuca - nearest hotel: El Limon Inn
ouani - nearest hotel: Al Amal
al wajh - nearest hotel: فندق رالين 2
lihue - nearest hotel: Kauai Palms
gunjur - nearest hotel: Footsteps Eco Lodge
crane - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
san julian - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
13,mecca,SA,21.4267,39.8261,64,Rehab Al Marwa Hotel 1
72,villa gonzalez ortega,MX,22.5000,-101.9167,15,No hotel found
127,holualoa,US,19.6228,-155.9522,77,Kona Hotel
201,mramani,KM,-12.3464,44.5286,83,Loulou Hotel
308,amayuca,MX,18.7000,-98.7667,25,El Limon Inn
343,ouani,KM,-12.1322,44.4258,79,Al Amal
372,al wajh,SA,26.2455,36.4525,60,فندق رالين 2
378,lihue,US,21.9789,-159.3672,65,Kauai Palms
430,gunjur,GM,13.2019,-16.7339,88,Footsteps Eco Lodge
444,crane,US,31.3974,-102.3501,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name","Country"]
    
    
    
)
    

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)